# Import Libraries

In [1452]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

import pandas as pd
import json
import csv
import nltk

import time
import re
from pyspark.sql.functions import split, explode
import numpy as np

# Login to Twitter

In [1453]:
#Open Twitter login page
driver = webdriver.Chrome("C:\\Users\\597705\\Downloads\\chromedriver.exe")
time.sleep(3)
driver.get(r"https://twitter.com/login")

In [1455]:
# Login to Twitter
username_field = driver.find_element_by_css_selector("#react-root > div > div > div.css-1dbjc4n.r-13qz1uu.r-417010 > main > div > div > div.css-1dbjc4n.r-13qz1uu > form > div > div:nth-child(6) > label > div > div.css-1dbjc4n.r-18u37iz.r-16y2uox.r-1wbh5a2.r-1udh08x > div > input")
username_field.send_keys('tenbrookchase@gmail.com')
time.sleep(.5)

username_field = driver.find_element_by_css_selector("#react-root > div > div > div.css-1dbjc4n.r-13qz1uu.r-417010 > main > div > div > div.css-1dbjc4n.r-13qz1uu > form > div > div:nth-child(7) > label > div > div.css-1dbjc4n.r-18u37iz.r-16y2uox.r-1wbh5a2.r-1udh08x > div > input")
username_field.send_keys('124Scrapy124!')
time.sleep(.5)

driver.find_element_by_css_selector("#react-root > div > div > div.css-1dbjc4n.r-13qz1uu.r-417010 > main > div > div > div.css-1dbjc4n.r-13qz1uu > form > div > div:nth-child(8) > div > div > span > span").click()

# Scrape Twitter Based on Key Word Search

We pursued another avenue of scraping twitter via the developer API, but for educational purposes we decided to use Selenium instead. The developer API also had limitations with the amount of pulls and the timeframe from which data could be accessed.

In [1489]:
# Change URL
def change_url(search_term):
    driver.get(r"https://twitter.com/search?q="+search_term+"&src=typed_query&f=live")
    return
change_url('mask')

In [ ]:
# Scrape tweet elements by CSS selector and store in the list "all_data"
page_count = 0
all_data = []

while page_count < 10000:
    page = driver.find_elements_by_css_selector('#react-root > div > div > div.css-1dbjc4n.r-18u37iz.r-13qz1uu.r-417010 > main')
    for job in page:
        all_data.append(job.text)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    page_count += 1
    time.sleep(.25)

In [1459]:
# Split the twitter data
splitter = []
for i in all_data:
    data = i.split('\n')
    splitter.append(data)

In [1461]:
flat_data = [item for sublist in splitter for item in sublist]

['Top',
 'Latest',
 'People',
 'Photos',
 'Videos',
 'Search timeline',
 'Pete Shop',
 '@PeteShop3',
 '·',
 'Nov 11',
 'Replying to ',
 '@zerohedge',
 'the question is if these people in COVID ICU units actually NEED to be there... cause I am hearing of people landing there who could just as well be sitting at home, sipping herbal tea.... we gotta keep this panic going you know.',
 'Alisa Nelson',
 '@alisagbrnelson',
 '·',
 'Nov 11',
 'Bass Pro Tournament of Champions canceled because of COVID-19; local tourney replacement',
 'Bass Pro Tournament of Champions canceled because of COVID-19; local tourney replacement',
 'Basketball fans will still be able to witness high caliber basketball teams competing Jan 14-16 during a one-time winter classic tournament which will feature four teams from Missouri.',
 'ky3.com',
 'SpinDoc',
 '@__iAhmadd',
 '·',
 'Nov 11',
 'Replying to ',
 '@SaraSar77212340',
 ' and ',
 '@covid_19_Arabic',
 'عموما مش ده الاساسي اللي احنا هنكون معتمدين عليه... واعتقد ب

In [1462]:
# Remove side panel data
page = []

for x in flat_data:
    if flat_data.index(x) > flat_data.index("Search timeline"):
        if flat_data.index(x) < flat_data.index("Search filters") or flat_data.index(x) > flat_data.index("© 2020 Twitter, Inc."):
            page.append(x)

In [1463]:
# Split the data into pages
data1 = [re.split('\\\n|Replying to \\\n|! |\\\\n| *[\.\?!][\'"\)\]]* *', i.strip('')) for i in page]
flat_list2 = [item for sublist in data1 for item in sublist]
page = list(filter(None, flat_list2))

In [1464]:
# Split the pages into tweets
tweets = []
for num in page:
    if num == '·':
        tweets.append(page[:page.index(num)-2])
        page = page[page.index(num)-2:]
        del page[page.index(num)]
#print(tweets)

In [1471]:
names = []
handles = []
sentences = []
tweets2 = []

for i in tweets:
    if(len(i)) > 4:
        name = i[0]
        handle = i[1]
        for sentence in i[3:]:
            names.append(name)
            handles.append(handle)
            sentences.append(sentence)
            tweets2.append(i)

In [1385]:
sentence2 = []
for v in sentences:
    for i in v:        
        sentence2.append(i)

['So did Bill Barr send his Federal Agents here to secure voting counts weren’t interfered with', 'Seems like this would be a good place to do that', 'Quote Tweet', 'FOX 10 Phoenix', '@FOX10Phoenix', ' · 31m', 'LIVE NOW: President Donald Trump supporters make their voices heard at the Arizona State Capitol and other state, county buildings as the election remains undecided on the night of Wednesday, Nov', '4', 'https://bit', 'ly/34YLK3u https://twitter', 'Replying to ', '@LepapillonBlu', 'Put on a mask, Karen', '4', '今バイデン側の投票がおかしいバイデン終わったなーってよく見るけど、トランプ側があえてそうやってる可能性とかないんかな？', 'Pointy End', 'Replying to ', '@davidfrum', ' and ', '@TheAtlantic', 'But it can be rebuilt, itll take time but it can', 'Replying to ', '@gableingaround', 'It’s just so', '2020', 'Replying to ', '@mask_passenger', 'あかんあかん！', 'あなたを擁護してるまさむね氏は「トランプ"圧勝"」って言うてますよ！', 'Replying to ', '@MuseZack', 'Not quite', 'The "It wasn\'t 52%', 'It was 47%', "crowd aren't quite being honest", 'Yes, after analysis, only 47% of wom

# Clean the data and convert to JSON for processing

In [1431]:
masks = pd.read_json(r'C:\Users\597705\Desktop\WIDS - COVID/masks.json', orient='records')

In [1436]:
masks.drop_duplicates(subset='Claim').shape

(491, 4)

In [1473]:
df = pd.DataFrame({'Claim': sentences, 'Name': names, 'Twitter Handle': handles, 'Tweet Text': tweets2})

In [1474]:
df1 = df
df1 = df1[df1.Claim.str.startswith("@") == False]
df1 = df1[df1.Claim.str.startswith(" @") == False]
df1 = df1[df1.Claim.str.startswith("Replying to") == False]
df1 = df1[df1.Claim.str.startswith(" and ") == False]
df1 = df1[df1['Claim'].map(len) >= 15]

In [1476]:
df1['Count'] = np.arange(len(df1))

In [1477]:
df1 = df1.set_index('Count')

In [1478]:
df1.head(25)

,Claim,Name,Twitter Handle,Tweet Text
Count,,,,
0,the question is if these people in COVID ICU u...,Pete Shop,@PeteShop3,"[Pete Shop, @PeteShop3, Nov 11, Replying to , ..."
1,cause I am hearing of people landing there who...,Pete Shop,@PeteShop3,"[Pete Shop, @PeteShop3, Nov 11, Replying to , ..."
2,we gotta keep this panic going you know,Pete Shop,@PeteShop3,"[Pete Shop, @PeteShop3, Nov 11, Replying to , ..."
3,Bass Pro Tournament of Champions canceled beca...,Alisa Nelson,@alisagbrnelson,"[Alisa Nelson, @alisagbrnelson, Nov 11, Bass P..."
4,Bass Pro Tournament of Champions canceled beca...,Alisa Nelson,@alisagbrnelson,"[Alisa Nelson, @alisagbrnelson, Nov 11, Bass P..."
5,Basketball fans will still be able to witness ...,Alisa Nelson,@alisagbrnelson,"[Alisa Nelson, @alisagbrnelson, Nov 11, Bass P..."
6,عموما مش ده الاساسي اللي احنا هنكون معتمدين عليه,SpinDoc,@__iAhmadd,"[SpinDoc, @__iAhmadd, Nov 11, Replying to , @S..."
7,واعتقد بلاد كتير لأنه محتاج تبريد عميق جدا وده...,SpinDoc,@__iAhmadd,"[SpinDoc, @__iAhmadd, Nov 11, Replying to , @S..."
8,Haan wasa ma asa soch sakta hun,Covid-20,@ZeeshanMunir165,"[Covid-20, @ZeeshanMunir165, Nov 11, Replying ..."


In [1479]:
df1.shape

(171617, 4)

# Keep sentences that contain assertions

In [1480]:
assertion_words_list = ["assert", "say", "proclaim", "profess", "attest", "predicat", "claim", "site", "alleg", "contend", "argue","mention", "defend", "declar", "insist", "challenge", "say", "express", "state", "think", "said", "know"]

def find_assertion(token_sentence):
    for word in assertion_words_list:
        if str(word) in token_sentence:
            return token_sentence
    return 'other'

In [1481]:
df1['Assertion'] = df1['Claim'].apply(lambda x: find_assertion(x))

In [1482]:
clean_df = df1.loc[df1['Assertion'] != 'other']

In [1483]:
clean_df.head()

,Claim,Name,Twitter Handle,Tweet Text,Assertion
Count,,,,,
2,we gotta keep this panic going you know,Pete Shop,@PeteShop3,"[Pete Shop, @PeteShop3, Nov 11, Replying to , ...",we gotta keep this panic going you know
15,A postal worker who claimed that his boss had ...,Sync Media Network,@joseph_ippolito,"[Sync Media Network, @joseph_ippolito, Nov 11,...",A postal worker who claimed that his boss had ...
34,"has said today, its because of COVID",Alex,@VOD88056105,"[Alex, @VOD88056105, Nov 11, Replying to , @CP...","has said today, its because of COVID"
42,Incidence data for states in US 2nd/3rd wave,john mcclean,@johnmcclean_ie,"[john mcclean, @johnmcclean_ie, Nov 11, Incide...",Incidence data for states in US 2nd/3rd wave
55,All you have done is whine about losing since ...,millertime,@millertime1955,"[millertime, @millertime1955, Nov 11, Replying...",All you have done is whine about losing since ...


In [1485]:
clean_df.to_json(r'C:\Users\597705\Desktop\WIDS - COVID\masks.json', orient = 'records')